In [1]:
#! pip install pyensae
#! pip install jyquickhelper

In [2]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Objectifs des prochaines séances

Connaissez-vous l'application [Pocket](https://getpocket.com/) ? C'est une application qui simplifie le bookmarking. Elle prend la forme d'une extension Chrome / Firefox. Quand on tombe sur un site intéressant, on peut le bookmaker, et ajouter, ou non, des tags pour "qualifier" le contenu.

Cette application répond au besoin de conserver le contenu web pertinent et de le classer.

Au cours des prochaines séances, nous allons construire un outil de machine learning qui :
- se connecte à un compte pocket
- récupère les sites bookmarqués et les tags éventuels
- à partir des articles taggés, prédit les meilleurs tags des sites non-taggés
- tag les articles non taggés

Bref, nous allons concevoir un programme de classification automatique des articles !

## Objectif de la séance

- Créer un compte Pocket
- S'authentifier auprès de l'API
- Populer le compte avec des données via l'API
- Récupérer les données via l'API
- Scraper les sites bookmarqués pour enrichir les données

## Mais c'est quoi une API ?

On vous explique tout ici :  
- [Définition](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#definition)
- [Les API qui existent](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#les-api-qui-existent)
- [Comment parler à une API ?](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#comment-parler-a-une-api)

Donc un API est une interface permettant de _communiquer_ avec une application. En général, on veut récupérer des données. Donc la communication consiste à envoyer une requete HTTP (le plus souvent GET ou POST) et à récupérer des données, souvent au format json. Ici nous voulons récupérer des données d'un compte utilisateur pocket. 

Pour savoir comment on communique précisément avec l'API de pocket, il n'y a pas de secret : il faut lire la doc de ceux qui l'ont codée. On vous a simplifié un peu les étapes ci-après.

## Création d'un compte Pocket

Créer un compte sur https://getpocket.com/signup?ep=4. Il n'y a pas de vérification d'email, donc vous pouvez mettre un faux mail.

Aller sur la _console developer_ de pocket: https://getpocket.com/developer/apps/index.php

<img src="./console_developer_pocket.png" width="450"/>

Cliquer sur CREATE AN APPLICATION

Compléter le formulaire comme suit (vous pouvez changer le nom de l'application et la description)

<img src="./screen_consumer_key.png" width="450"/>

Cliquer sur CREATE APPLICATION

In [22]:
CONSUMER_KEY = input("Insérer ici le CONSUMER KEY de la plateform WEB ")

Insérer ici le CONSUMER KEY de la plateform WEB 71236-7827e5507cf71bc311069b3a


<img src="./screen_consumer_key2.png" width="450"/>

Vous en aurez besoin pour vous connecter à l'API de pocket.

## Authentification

D'abord il faut s'[authentifier](https://getpocket.com/developer/docs/authentication)

Protocole utilisé ici : [OAUTH2](https://tools.ietf.org/html/rfc6749) (très classique). 

<img src="./screen_oauth2.png"/>

6 étapes donc avant d'avoir le droit de récupérer les données. De temps en temps, il existe une librairie python. C'est notre cas : https://github.com/tapanpandita/pocket. On va s'en servir pour s'authentifier. Mais pas pour récupérer les données (elle n'est plus à jour pour faire ça).

In [4]:
! pip install pocket

You are using pip version 9.0.0, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Etape 1 : Obtenir un code d'authorisation => get_request_token

In [23]:
import pocket
from pocket import Pocket

REDIRECT_URI = "http://localhost:8888/notebooks/API%20Pocket.ipynb"
# c'est l'url à laquelle vous allez rediriger l'utilisateur (ici, vous) après que pocket a authentifié l'utilisateur (vous)
REQUEST_TOKEN = Pocket.get_request_token(consumer_key=CONSUMER_KEY, redirect_uri=REDIRECT_URI)
print(REQUEST_TOKEN)

519d719c-e778-e6e6-6ac8-c7ff9e


### Etape 2: Authoriser l'accès

In [24]:
url = "https://getpocket.com/auth/authorize?request_token={0}&redirect_uri={1}".format(REQUEST_TOKEN, REDIRECT_URI)
print("Aller à l'url : \n" + url)

Aller à l'url : 
https://getpocket.com/auth/authorize?request_token=519d719c-e778-e6e6-6ac8-c7ff9e&redirect_uri=http://localhost:8888/notebooks/API%20Pocket.ipynb


<img src="./screen_authorization_pocket.png" width="450"/>

Cliquer sur Autoriser.

### Etape 3: Récupérer le token d'accès

In [25]:
USER_CREDENTIALS = Pocket.get_credentials(consumer_key=CONSUMER_KEY, code=REQUEST_TOKEN)
print(USER_CREDENTIALS)

{'username': 'elodie.royant@tagerstreet.com', 'access_token': 'f58a4aa3-85e2-ffe1-3c66-6e1d45'}


In [26]:
ACCESS_TOKEN = USER_CREDENTIALS['access_token']
print(ACCESS_TOKEN)

f58a4aa3-85e2-ffe1-3c66-6e1d45


## Chargement de données sur le nouveau compte

Comme vous venez de créer un compte, vous n'avez pas encore d'articles sauvegardés. On vous a préparé un peu moins de 500 articles (format json). Un tiers de ces articles sont taggés (catégorisés). Un article peut comprendre un ou plusieurs tags.

On vous rappelle que l'objectif à termes sera de prédire les meilleurs tags pour les articles non taggés, étant donnés les mots qui caractérisent ces articles (titre, résumé, et ensemble des mots présents dans le html de la page).

### Chargement du fichier json en mémoire

In [10]:
import json
from pprint import pprint

with open('./data_pocket.json') as fp:    
    data = json.load(fp)

pprint(data)

{'1003565100': {'authors': {'55834601': {'author_id': '55834601',
                                         'item_id': '1003565100',
                                         'name': 'Grafikart.fr',
                                         'url': ''}},
                'excerpt': 'Ionic est un framework qui va vous permettre de '
                           'créer des applications mobiles en utilisant des '
                           'technologies Web. Ionic se base pour cela sur '
                           "d'autres frameworks / technologies qui ont fait "
                           'leurs preuves.  Avant de pouvoir commencer, il '
                           'nous faut évidemment commencer par installer '
                           "l'outil.",
                'favorite': '0',
                'given_title': 'Tutoriel Vidéo Apache Cordova Ionic Framework',
                'given_url': 'https://www.grafikart.fr/tutoriels/cordova/ionic-framework-641',
                'has_image': '0',
      

### Exercice 1 : chargement des données du json dans le compte nouvellement créé 

Pour communiquer avec une API, il faut envoyer des requêtes HTTP. Ici on veut ajouter les données du fichier json ("given_url" et "tags") dans le compte Pocket.

Que nous dit la doc ? Consulter https://getpocket.com/developer/docs/v3/add.

Par exemple, pour l'ajout d'un seul item, la doc nous donne l'url à laquelle il faut envoyer une requête (https://getpocket.com/v3/add), et la méthode qu'il faut employer. Ici, il s'agit d'une méthode [POST](https://en.wikipedia.org/wiki/POST_(HTTP)).

Pour envoyer une requête en python, il y a plusieurs solutions. Un des plus simples consiste à utiliser la librairie [requests](http://docs.python-requests.org/en/master/user/quickstart/#make-a-request).

A vous de jouer ! Commencez par un seul (par exemple http://docs.python-requests.org/en/master/user/quickstart/ avec les tags "python, requests"), puis si cela a fonctionné, vous pouvez passer 100 items (pas plus, le maximum de requêtes autorisées par l'API est de 500 par heure et par utilisateur).

Pour voir si cela a marché, il suffit d'aller sur votre compte Pocket :)

### Exercice 1 - correction 

#### Ajouter un seul item

In [11]:
import requests

data_test = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"url":"http://docs.python-requests.org/en/master/user/quickstart/",
"tags": "python, requests"}

one_item = requests.post('https://getpocket.com/v3/add', data = data_test)

#### Ajouter l'ensemble des items

In [13]:
from requests.utils import quote

list_action_add = []
for k,v in data.items():
    if  'tags' in v:
        list_action_add.append({'consumer_key':CONSUMER_KEY, 
                                'access_token':ACCESS_TOKEN,
                                'url': v['given_url'], #sert à encoder les urls
                                'tags': ','.join(list(v['tags'].keys()))
                               })
list_action_add[0:100]

[{'access_token': '9887c3b4-2942-ad11-06b7-f08251',
  'consumer_key': '71275-41682fe6e8b4cc26c7451fd3',
  'tags': 'parkingspot',
  'url': 'https://code.tutsplus.com/courses/custom-interactive-maps-with-the-google-maps-api'},
 {'access_token': '9887c3b4-2942-ad11-06b7-f08251',
  'consumer_key': '71275-41682fe6e8b4cc26c7451fd3',
  'tags': 'react',
  'url': 'https://developers.google.com/web/fundamentals/engage-and-retain/web-app-manifest/'},
 {'access_token': '9887c3b4-2942-ad11-06b7-f08251',
  'consumer_key': '71275-41682fe6e8b4cc26c7451fd3',
  'tags': 'ssh',
  'url': 'http://sebastien.saunier.me/blog/2015/05/10/github-public-key-authentication.html'},
 {'access_token': '9887c3b4-2942-ad11-06b7-f08251',
  'consumer_key': '71275-41682fe6e8b4cc26c7451fd3',
  'tags': 'social,social network,geek',
  'url': 'https://www.awwwards.com/10-social-networks-for-developers.html'},
 {'access_token': '9887c3b4-2942-ad11-06b7-f08251',
  'consumer_key': '71275-41682fe6e8b4cc26c7451fd3',
  'tags': 'new,

In [19]:
c = 0

In [ ]:
import time

for item in list_action_add[c:]:
    time.sleep(2)
    try:
        requests.post('https://getpocket.com/v3/add', data = item)
    except:
        "Retry, api rejected item #{0}".format(c)
    c+=1

## Récupération des données

### Exercice 2 : récupérer les urls et les tags des items qui contiennent le tag "python"

C'est par ici : https://getpocket.com/developer/docs/v3/retrieve. A vous de jouer !

### Exercice 2 - correction

In [27]:
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"tag": "python",
"detailType":"complete"}

exo2_items = requests.post('https://getpocket.com/v3/get', data = items)
exo2_list = [v for k,v in exo2_items.json()['list'].items()]

In [28]:
[v['given_url'] for k,v in exo2_items.json()['list'].items()]

['http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/td_2a.html',
 'https://qbox.io/blog/building-an-elasticsearch-index-with-python',
 'https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/',
 'http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/all_notebooks_coverage.html',
 'https://pythonprogramming.net/',
 'http://www.nltk.org/book/ch03.html',
 'https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/',
 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html',
 'http://blog.fouadhamdi.com/introduction-a-nltk/',
 'http://www.xavierdupre.fr/app/mlstatpy/helpsphinx/index.html#mlstatpy',
 'https://blog.rstudio.org/2016/03/29/feather/',
 'https://marcobonzanini.com/2015/06/16/mining-twitter-data-with-python-and-js-part-7-geolocation-and-interactive-maps/',
 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/index.html#teachpyx',
 'http://nbviewer.jupyter.org/github/ptwobrussell/Mining-the-Social-Web-2nd-Edition/tree/master/ipynb/

In [29]:
[', '.join(list(v['tags'].keys())) for k,v in exo2_items.json()['list'].items()]

['python, cours, ensae',
 'python, elastic-search',
 'python',
 'python, cours, ensae',
 'python',
 'python, tokenize, nltk',
 'python, elastic-search, elasticsearch, tutorial',
 'python, cours, ensaen',
 'python, french, tokenize, tokenizer, nltk, nlp',
 'python, cours, ensae',
 'python',
 'python, tagerstreet',
 'python, cours, ensae',
 'python',
 'python, cours, ensae',
 'python, nlp']

### Exercice 3 : récupérer les urls et les titres des items qui contiennent le mot "python" dans le titre ou l'url

### Exercice 3 - correction

In [31]:
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"search": "python",
"detailType":"complete"}

exo3_items = requests.post('https://getpocket.com/v3/get', data = items)
exo3_list = [v for k,v in exo3_items.json()['list'].items()]

In [32]:
[v['given_url'] for k,v in exo3_items.json()['list'].items()]

['http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/td_2a.html',
 'https://qbox.io/blog/building-an-elasticsearch-index-with-python',
 'https://jakevdp.github.io/blog/2015/08/14/out-of-core-dataframes-in-python/',
 'http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/all_notebooks_coverage.html',
 'https://pythonprogramming.net/',
 'https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/',
 'https://realpython.com/blog/python/setting-up-sublime-text-3-for-full-stack-python-development/',
 'https://blog.dominodatalab.com/ab-testing-with-hierarchical-models-in-python/',
 'https://www.digitalocean.com/community/tutorials/how-to-install-the-anaconda-python-distribution-on-ubuntu-16-04',
 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html',
 'https://pypi.python.org/pypi/munkres/',
 'https://people.duke.edu/~ccc14/sta-663/Optimization_Bakeoff.html',
 'https://blog.rstudio.org/2016/03/29/feather/',
 'http://stackoverflow.com/questions/966391

In [33]:
[v['resolved_title'] for k,v in exo3_items.json()['list'].items()]

['Python pour un Data Scientist / Economiste¶',
 'Build an Elasticsearch Index with Python—Machine Learning Series, Part 1',
 'Out-of-Core Dataframes in Python: Dask and OpenStreetMap',
 'Notebooks Coverage¶',
 'Python Programming Tutorials',
 'Python + Elasticsearch. First steps.',
 'Setting Up Sublime Text 3 for Full Stack Python Development',
 'A/B Testing with Hierarchical Models in Python',
 'How To Install the Anaconda Python Distribution on Ubuntu 16.04',
 'Types et variables du langage python¶',
 'munkres 1.0.9',
 'Optimization bake-off¶',
 'Feather: A Fast On-Disk Format for Data Frames for R and Python, powered by Apache Arrow',
 'How can I tag and chunk French text using NLTK and Python?',
 'Mining Twitter Data with Python (and JS) – Part 7: Geolocation and Interactive Maps',
 'Apprendre la programmation avec Python¶',
 'Interpreting A/B Test using Python',
 'Galleries de notebooks¶',
 'sdpython/ensae_teaching_cs',
 'Python NLTK WTF, Chapter 1: Notes on things that don’t wor

Dans le fichier qui servira à la catégorisation automatique, nous allons avoir besoin : de l'url, du titre, de l'extrait, des tags et du contenu.
    
L'api ne permet pas d'accéder au contenu des sites épinglés. Mais nous pouvons le récupérer grâce à l'url.

### Exercice 4 : constituer un DataFrame avec les champs dont nous allons avoir besoin, sauf le contenu

Constituer d'abord un DataFrame avec les champs resolved_url, resolved_title, excerpt, et les tags des items comprenant le terme python.

In [34]:
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"detailType":"complete"}

exo4_items = requests.post('https://getpocket.com/v3/get', data = items)
len(exo4_items.json()['list'])

477

In [35]:
exo4_list = [v for k,v in exo4_items.json()['list'].items() if 'Python' in v['excerpt']]

In [36]:
import pandas as p

df2 = p.DataFrame(exo2_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]
df3 = p.DataFrame(exo3_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]
df4 = p.DataFrame(exo4_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]

In [40]:
import numpy as np
df = p.merge(df2, df3, on='item_id', how='outer', suffixes=('', '_df3'))
df = p.merge(df,  df4, on='item_id', how='outer', suffixes=('', '_df4'))

def complete_data(x,y,z):
    if x != x:
        if y != y:
            return z
        else:
            return y
    else:
        return x

df['url'] = np.vectorize(complete_data)(df['resolved_url'], df['resolved_url_df3'], df['resolved_url_df4'])
df['title'] = np.vectorize(complete_data)(df['resolved_title'], df['resolved_title_df3'], df['resolved_title_df4'])
df['excerpt'] = np.vectorize(complete_data)(df['excerpt'], df['excerpt_df3'], df['excerpt_df4'])
df['tags'] = np.vectorize(complete_data)(df['tags'], df['tags_df3'], df['tags_df4'])
df = df[['item_id','url','title','excerpt', 'tags']]
df['tags'] = df['tags'].apply(lambda x: ','.join(list(x.keys())) if x==x else x)
df

item_id                                                url  \
0    934060680  http://www.xavierdupre.fr/app/ensae_teaching_c...   
1   1064802343  https://qbox.io/blog/building-an-elasticsearch...   
2    378831480  https://jakevdp.github.io/blog/2013/06/15/numb...   
3   1883858743  http://www.xavierdupre.fr/app/ensae_teaching_c...   
4    680797791                     https://pythonprogramming.net/   
5    241420475                 http://www.nltk.org/book/ch03.html   
6   1395948696  https://tryolabs.com/blog/2015/02/17/python-el...   
7   1883956314  http://www.xavierdupre.fr/app/teachpyx/helpsph...   
8   1057169119    http://blog.fouadhamdi.com/introduction-a-nltk/   
9   1473942596  http://www.xavierdupre.fr/app/mlstatpy/helpsph...   
10  1240472260       https://blog.rstudio.org/2016/03/29/feather/   
11   957402029  https://marcobonzanini.com/2015/06/16/mining-t...   
12  1665628656  http://www.xavierdupre.fr/app/teachpyx/helpsph...   
13  1056127688  http://nbviewer.jupyter.org/github/ptwobrussel...   
14  1870404087      https://github.com/sdpython/ensae_teaching_cs   
15  1072497525  http://www.mikesboyle.com/post/117202964694/py...   
16  1011618630  https://jakevdp.github.io/blog/2015/08/14/out-...   
17   687517654  https://realpython.com/blog/python/setting-up-...   
18  1014684096  https://blog.dominodatalab.com/ab-testing-with...   
19  1582635439  https://www.digitalocean.com/community/tutoria...   
20   833287792              https://pypi.python.org/pypi/munkres/   
21   952475069  https://people.duke.edu/~ccc14/sta-663/Optimiz...   
22   295141437  http://stackoverflow.com/questions/9663918/how...   
23   829788447            http://okomestudio.net/biboroku/?p=2375   
24  1893141176  http://www.xavierdupre.fr/app/ensae_teaching_c...   
25   163291612  https://blog.miguelgrinberg.com/post/the-flask...   
26   241420549                 http://www.nltk.org/book/ch04.html   

                                                title  \
0         Python pour un Data Scientist / Economiste¶   
1   Build an Elasticsearch Index with Python—Machi...   
2                            Numba vs. Cython: Take 2   
3                                 Notebooks Coverage¶   
4                        Python Programming Tutorials   
5                               3 Processing Raw Text   
6                Python + Elasticsearch. First steps.   
7               Types et variables du langage python¶   
8   Introduction à l'analyse de texte avec nltk - ...   
9   Les maths d’abord, la programmation ensuite — ...   
10  Feather: A Fast On-Disk Format for Data Frames...   
11  Mining Twitter Data with Python (and JS) – Par...   
12            Apprendre la programmation avec Python¶   
13                            Jupyter Notebook Viewer   
14                         sdpython/ensae_teaching_cs   
15  Python NLTK WTF, Chapter 1: Notes on things th...   
16  Out-of-Core Dataframes in Python: Dask and Ope...   
17  Setting Up Sublime Text 3 for Full Stack Pytho...   
18     A/B Testing with Hierarchical Models in Python   
19  How To Install the Anaconda Python Distributio...   
20                                      munkres 1.0.9   
21                             Optimization bake-off¶   
22  How can I tag and chunk French text using NLTK...   
23                 Interpreting A/B Test using Python   
24                            Galleries de notebooks¶   
25     The Flask Mega-Tutorial, Part I: Hello, World!   
26                                                      

                                              excerpt  \
0   Bibliographie¶ Livres sur le machine learning ...   
1   In this first article, we're going to set up s...   
2   Last summer I wrote a post comparing the perfo...   
3                                                       
4   Learn how to use Python with Pandas, Matplotli...   
5   The most important source of texts is undoubte...   
6   Lately, here at Tryolabs, we started gaining i...   
7   Il est impossible